In [1]:
import IO_map
import random
import re
import generate_verilog as gv

"""
Wire can be in many groups: A wire can be on many linear paths
    - Every wire is initally in a group with it's parent and child modules
Wire can be in many loops: A wire can be on many looped paths
Groups contains {group_id : [modules in that group]}
    - Initially one group for every input and output pair 
    - Group id is initially the wire id, since each wire is in it's own group
Loops contains {loop_id : [modules in that loop]}

"""
class Fuzz_Run:
    def __init__(self, flattened_lib_path):
        random.seed(0)
        self.flattened_lib_path = flattened_lib_path
        self.paths, self.port_to_module, io_map = IO_map.create_IO_map(flattened_lib_path)
        self.all_wires = io_map['bit_input']
        self.all_wires.update(io_map['bit_output']) # Single source of truth
        self.external_inputs = list(io_map['bit_input'].keys())
        self.external_outputs = list(io_map['bit_output'].keys()) # Unchanged will always output all outputs
        self.internal_connections = []
        # self.top_module = gv.generate_top_module(self.io_map, [self.internal_outputs, self.internal_inputs], self.external_inputs, self.external_outputs)
    
        self.wire_loops = {}
    
    def linear_rewire(self):
        """
        1. Need to choose a output wire that is not associated with any loops
        2. Need to choose a input wire that is not associated with any loops
            - Wire also cannot be in the group that the output wire is in.
        """
        # Takes a current layout, stacks module without creating a cycle
        output_wire_id = random.choice(self.external_outputs)
        output_wire = self.all_wires[output_wire_id]

        # Need to choose a wire from a module that is not in
        available_path_ids = list(self.paths.keys() - output_wire.paths)
        if len(available_path_ids) == 0:
            return "No linear rewiring available, nothing done"
            
        chosen_path_id = random.choice(available_path_ids)
        chosen_input_wire_id = random.choice(list(self.paths[chosen_path_id]))
        chosen_input_wire = self.all_wires[chosen_input_wire_id]
        chosen_input_wire_port = random.choice(chosen_input_wire.output)

        # Add new wire connection 
        output_wire.output.append(chosen_input_wire_port)

        # Disconnect the wire's output
        chosen_input_wire.output.remove(chosen_input_wire_port)


        # Merge paths

        # Add to wire to internal_connections

        # If this wire recieves its data from ext input, then we need to remove it
        # if its not outputting to any ports. Otherwise its receiving it's data from
        # a module, and will always be connected to the ext output.
        if chosen_input_wire.type == IO_map.Wire.Wire_Type.ITM and len(chosen_input_wire.output) == 0:
            self.all_wires.pop(chosen_input_wire.id)
            self.external_inputs.remove(chosen_input_wire.id)

In [ ]:
test_fuzz = Fuzz_Run("/module-fuzz/test_library/flattened_IO")
test_fuzz.linear_rewire()